In [ ]:
"""
Created on Fri Feb 19 08:46:16 2021

@author: Schroeder
"""

# Liest holc etc. daten von bereits auf platte befindlichen csv dataframes .
# Soll Plot und Auswertung von datagrabbing trennen.

import plotly.io as pio
import plotly.express as px
from plotly.figure_factory import create_candlestick
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot

import TickerSelector, regression
from datetime import datetime
from datetime import date
import calendar

# Imports
from pandas_datareader import data as pdr
#from yahoo_fin import stock_info as si

import numpy as np
import scipy.stats as stats


from pandas import ExcelWriter
import yfinance as yf
import pandas as pd
import datetime
import time
import csv
   
#import StockscreenerPlotter_stats
#import StockscreenerWinners_stats
#import downloader

import sys
import os.path
from os.path import isfile, join

from os import listdir
#from os.path import isfile, join

#currentdir = os.path.join(os.path.dirname(__file__))

###### Insert the following three lines to make any import lib in he project dir setup visible to an other
###### Directory in the project setup

currentdir = os.path.abspath('')
parentdir = os.path.realpath(os.path.join(currentdir, '..'))
sys.path.insert(0, parentdir) 
#############################################################

from lib import Indikatoren

#lieferrt bei gegebenem datatfram mit datum als index das erste Datum, das letzte, und zwei 
# dazwischen


    

## Gebe Pfad an, unter dem das Datenfile zu finden ist.
## Bsp: C:/Users/Schroeder/OneDrive/Trading/ETFS/RES_Aristokraten/Res/


# --------------------------------------------------------------------------
## Welcher Sektor oder index soll analysieert werden ?

In [ ]:
pfad = os.path.realpath(os.path.join(parentdir, '..'))
pfad = os.path.realpath(os.path.join(pfad, 'ETFS'))

#############################################################
who = "RES_SP500"
pfad = os.path.realpath(os.path.join(pfad, who))
###############################################################
pfad = os.path.realpath(os.path.join(pfad, 'Data'))


file  = "_ticker_names.csv"

singleticker_file=os.path.realpath(os.path.join(pfad, file))



In [19]:
exportList = pd.DataFrame(columns=['ticker',"z-score","MACDsignal","RSIlevel","Name","Industry","Sector","marketCap"])
mydict={}

## ---------------------------------------------------------------------

In [20]:
# DAs ist der Ticker fiel, der alle Ticker enthaelt, die gelsen werden:
tkr = pd.read_csv(singleticker_file,sep=";",decimal=',',parse_dates=True,index_col=0)
        #df = yf.download(ticker, start=start_date.strftime("%Y-%m-%d"), end=end_date.strftime("%Y-%m-%d"))
    
for i in tkr["ticker"]:
    file=pfad+ "/"  + i + ".csv"
    df = pd.read_csv(file,sep=";",decimal=',',parse_dates=True,index_col=0)
    #print(file)
    df["dummy"]= df["Factor"]-df["EMA100"]
    df["dummy"]=df["dummy"].fillna(0)
    Indikatoren.Zscore_rolling(df,"dummy",100)
    #df["Zscore"]=stats.zscore(df["dummy"])
    #Prüfe auf z-score
    if (df["z-score"][-1] < -1.5) or (df["z-score"][-1] > 1.5):
        mydict["ticker"]=i
        mydict["z-score"]=df["z-score"][-1]
        mydict["Name"]=tkr["name"][tkr["ticker"]==i]
        
    if((df["MACDdif_week"][-1] > 0) and \
        ((df["MACD_week"][-1] > 0) and (df["MACDsign_week"][-1] < 0))):
            mydict["MACDsignal"]="Long"
    if((df["MACDdif_week"][-1] < 0) and \
        ((df["MACD_week"][-1] < 0) and (df["MACDsign_week"][-1] > 0))):
            mydict["MACDsignal"]="Short"        
    exportList =exportList.append(mydict,ignore_index=True) 
    
    if i == "HRB":
        fig = px.line(df, x=df.index, y=[df["MACD_week"],df["MACDsign_week"]])
        fig.show()   
#exportList.style.background_gradient()
print(exportList.sort_values(by="z-score",ascending=False))


    ticker   z-score MACDsignal  RSIlevel  \
70       C  2.927083      Short       NaN   
71       C  2.927083      Short       NaN   
72       C  2.927083      Short       NaN   
199     HD  2.849977      Short       NaN   
200     HD  2.849977      Short       NaN   
..     ...       ...        ...       ...   
26     ALL -2.022063      Short       NaN   
65     BMY -2.212416      Short       NaN   
217    IBM -2.357312      Short       NaN   
218    IBM -2.357312      Short       NaN   
219    IBM -2.357312      Short       NaN   

                                                  Name  Industry  Sector  \
70     102    Citigroup Inc.
Name: name, dtype: object       NaN     NaN   
71     102    Citigroup Inc.
Name: name, dtype: object       NaN     NaN   
72     102    Citigroup Inc.
Name: name, dtype: object       NaN     NaN   
199  221    The Home Depot, Inc.
Name: name, dtype:...       NaN     NaN   
200  221    The Home Depot, Inc.
Name: name, dtype:...       NaN     NaN   
.. 